# Deploying A Sentence-Embedding Model

## Contents

1. [Setup](#Setup)
2. [Deploy](#Deploy)
3. [Evaluate](#Deploy)

# Setup

- Load environmental variables
- Create Sagemaker Session
- Define S3 Bucket, path and model name
- Provide IAM role

In [11]:
%load_ext dotenv
import os
import sagemaker
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer
from sagemaker.pytorch import PyTorchModel

%dotenv -v

bucket = os.getenv("S3_BUCKET")
model_location = os.getenv("MODEL_PATH")   # path to model data
entry = os.getenv("ENTRY_POINT")     # Python file to execute
source = os.getenv("SOURCE_DIR")       # Source for code
version = os.getenv("FRAMEWORK")           # Pytorch Version
role = sagemaker.get_execution_role()      # IAM role

git_config = os.getenv("GIT_CONFIG")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


# Deploy

Provide custom hosting functions:
- input_fn: Deserializes input data
- model_fn: Loads pytorch model
- predict_fn: run inference with input data
- output_fn: Serializes results of predict_fn
- predictor_cls: Creates the predictor with the endpoint name

In [4]:
class JSONPredictor(RealTimePredictor):
    """Serializes and deserializes json
    """
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [7]:
model = PyTorchModel(model_data=model_location,
                     role=role,
                     framework_version=version,
                     entry_point=entry,
                     source_dir=source,
                     predictor_cls=JSONPredictor,
                     git_config=git_config)

TypeError: string indices must be integers

In [ ]:
predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.m5.xlarge',
                         endpoint_name="sentence_similarity")

In [10]:
print(git_config)

{'repo': 'https://github.com/sam3ay/sentence-compare.git', 'branch': 'code'}


# Evaluate

Determine Textual Similarity